In [ ]:
!pip install ta

In [ ]:
import yfinance as yf
import pandas as pd
import requests
from bs4 import BeautifulSoup
from datetime import datetime, timedelta
import ta
 
def get_all_tickers():
    tickers = set()
 
    # S&P 500
    try:
        url_sp500 = "https://en.wikipedia.org/wiki/List_of_S%26P%20500_companies"
        response = requests.get(url_sp500)
        soup = BeautifulSoup(response.text, "lxml")
        table = soup.find("table", {"id": "constituents"})
        for row in table.find_all("tr")[1:]:
            cells = row.find_all("td")
            if len(cells) > 1:
                symbol = cells[0].text.strip().replace(".", "-")
                if symbol:
                    tickers.add(symbol)
    except Exception as e:
        print("Errore caricamento S&P 500:", e)
 
    # Nasdaq 100
    try:
        url_nasdaq100 = "https://en.wikipedia.org/wiki/NASDAQ-100"
        response = requests.get(url_nasdaq100)
        soup = BeautifulSoup(response.text, "lxml")
        table = soup.find("table", {"id": "constituents"})
        for row in table.find_all("tr")[1:]:
            cells = row.find_all("td")
            if len(cells) > 1:
                symbol = cells[0].text.strip().replace(".", "-")
                if symbol:
                    tickers.add(symbol)
    except Exception as e:
        print("Errore caricamento Nasdaq 100:", e)
 
    # DAX
    try:
        url_dax = "https://en.wikipedia.org/wiki/DAX"
        response = requests.get(url_dax)
        soup = BeautifulSoup(response.text, "lxml")
        table = soup.find("table", {"id": "constituents"})
        for row in table.find_all("tr")[1:]:
            cells = row.find_all("td")
            if len(cells) > 1:
                symbol = cells[0].text.strip().replace(".", "-")
                if symbol:
                    tickers.add(symbol)
    except Exception as e:
        print("Errore caricamento DAX:", e)
 
    # FTSE MIB
    ftse_mib_tickers = [
        "A2A.MI", "AMP.MI", "AZM.MI", "BMED.MI", "BMPS.MI", "BAMI.MI", "BPSO.MI", "BPE.MI", "BC.MI", "BZU.MI",
        "CPR.MI", "DIA.MI", "ENEL.MI", "ENI.MI", "RACE.MI", "FBK.MI", "G.MI", "HER.MI", "IP.MI", "ISP.MI",
        "INW.MI", "IG.MI", "IVG.MI", "LDO.MI", "MB.MI", "MONC.MI", "NEXI.MI", "PIRC.MI", "PST.MI", "PRY.MI",
        "REC.MI", "SPM.MI", "SRG.MI", "STLAM.MI", "STMMI.MI", "TIT.MI", "TEN.MI", "TRN.MI", "UCG.MI", "UNI.MI"
    ]
    tickers.update(ftse_mib_tickers)
 
    # CAC 40
    cac40_tickers = [
        "AC.PA", "AI.PA", "AIR.PA", "MT.AS", "CS.PA", "BNP.PA", "EN.PA", "CAP.PA", "CA.PA", "ACA.PA",
        "BN.PA", "DSY.PA", "EDEN.PA", "ENGI.PA", "EL.PA", "ERF.PA", "RMS.PA", "KER.PA", "OR.PA", "LR.PA",
        "MC.PA", "ML.PA", "ORA.PA", "RI.PA", "PUB.PA", "RNO.PA", "SAF.PA", "SGO.PA", "SAN.PA", "SU.PA",
        "GLE.PA", "STLAP.PA", "STMPA.PA", "TEP.PA", "HO.PA", "TTE.PA", "URW.PA", "VIE.PA", "DG.PA", "VIV.PA"
    ]
    tickers.update(cac40_tickers)
 
    # Extra
    extra = [
        "CL=F", "GC=F", "SI=F", "NG=F", "HG=F", "PA=F", "PL=F", "KC=F", "CC=F",
        "XLC", "XLY", "XLP", "XLE", "XLF", "XLV", "XLI", "XLB", "XLRE", "XLK", "XLU",
        "000300.SS", "^HSI", "^N225", "^GDAXI", "FTSEMIB.MI", "^IXIC", "^GSPC", "^DJI", "^RUT"
    ]
    tickers.update(extra)
 
    return sorted(tickers)
 
def analyze_key_reversal(tickers):
    lookback = 2
    rsi_period = 9
    cutoff_date = datetime.today() - timedelta(days=30)
    results = []
 
    for ticker in tickers:
        try:
            df = yf.download(ticker, period="2y", interval="1wk", progress=False, group_by='ticker')
            if df.empty:
                continue
 
            if isinstance(df.columns, pd.MultiIndex):
                df.columns = df.columns.get_level_values(1)
            elif any(ticker in col for col in df.columns):
                df.columns = [col.split('.')[-1] for col in df.columns]
 
            df.index = pd.to_datetime(df.index)
            df["RSI"] = ta.momentum.RSIIndicator(close=df["Close"], window=rsi_period).rsi()
            df["Close_1"] = df["Close"].shift(1)
            df["Low_1n"] = df["Low"].shift(1).rolling(lookback).min()
            df["High_1n"] = df["High"].shift(1).rolling(lookback).max()
 
            df["KR_Up"] = (df["Low"] < df["Low_1n"]) & (df["Close"] > df["Close_1"]) & (df["RSI"] < 30)
            df["KR_Down"] = (df["High"] > df["High_1n"]) & (df["Close"] < df["Close_1"]) & (df["RSI"] > 70)
 
            signals = df[(df["KR_Up"]) | (df["KR_Down"])].copy()
            signals = signals[signals.index >= cutoff_date]
 
            for date, row in signals.iterrows():
                results.append({
                    "Ticker": ticker,
                    "Date": (pd.to_datetime(date) + timedelta(days=4)).strftime("%Y-%m-%d"),
                    "Signal": "Rialzista" if row["KR_Up"] else "Ribassista"
                })
 
        except Exception as e:
            # Se vuoi, puoi commentare la riga qui sotto per non stampare errori
            print(f"Errore su {ticker}: {e}")
 
    df_out = pd.DataFrame(results)
    if not df_out.empty:
        print(df_out[["Ticker", "Date", "Signal"]])
    else:
        print("No signals found within the specified date range.")
 
    return df_out
 
if __name__ == "__main__":
    all_tickers = get_all_tickers()
    analyze_key_reversal(all_tickers)
 

In [ ]:
from datetime import datetime

# Get the current week number
week_number = datetime.now().isocalendar()[1]

# Define the output filename
output_filename = f"key_reversal_week_{week_number}.xlsx"

# Use the results from the previous execution
# df_results = analyze_key_reversal(get_all_tickers()) # Removed this line

# Save the results to an Excel file, overwriting if it exists
if not df_results.empty:
    df_results.to_excel(output_filename, index=False, sheet_name="Key Reversal")
    print(f"Results saved to {output_filename}")
else:
    print("No results to save.")